In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from scipy import stats
from scipy.special import boxcox, inv_boxcox
import optuna
from optuna.samplers import TPESampler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler 

In [14]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score


In [2]:
df=pd.read_csv("train.csv")

In [13]:
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700


In [20]:
# df_temp=df[["Outlet_Type","Item_Outlet_Sales"]]
# df_grp_ot=df_temp.groupby("Outlet_Type").mean().reset_index().sort_values("Item_Outlet_Sales").reset_index()
# df["Outlet_Type"]=df["Outlet_Type"].replace(dict(zip(df_grp_ot["Outlet_Type"],df_grp_ot["index"])))
# df["Outlet_Type"]=df["Outlet_Type"].astype("int32")
# df["Outlet_Type"].value_counts()

In [49]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [48]:
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0], inplace=True)


In [36]:
# df_test=df[["Outlet_Size","Item_Outlet_Sales"]]
# df_grp=df_test.groupby(["Outlet_Size"]).mean()
# df_grp.sort_values(by=['Item_Outlet_Sales'],inplace=True,ascending=True)
# ax=df_grp.plot.bar()

In [4]:
df_temp=df.groupby(["Outlet_Size"]).mean()

In [5]:
df_temp

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Outlet_Size,,,,,
High,13.006148,0.059957,141.425982,1987.000000,2298.995256
Medium,12.895420,0.060142,140.590514,1997.635875,2681.603542
Small,12.848075,0.071383,141.970209,1997.072864,1912.149161


In [21]:
#cat=df_temp.groupby(["Item_Type"])["Item_Outlet_Sales"].mean().reset_index().sort_values(by=['Item_Outlet_Sales'],inplace=True,ascending=True)


In [3]:
df_temp=df[["Item_Type","Item_Outlet_Sales"]]
df_grp=df_temp.groupby("Item_Type").mean().reset_index().sort_values("Item_Outlet_Sales").reset_index()

df_temp=df[["Outlet_Type","Item_Outlet_Sales"]]
df_grp_ot=df_temp.groupby("Outlet_Type").mean().reset_index().sort_values("Item_Outlet_Sales").reset_index()

def preprocessing2(df2):
    # remove null values in Item_Weight
    df2['Item_Weight']=df2.groupby('Item_Type')['Item_Weight'].apply(lambda x:x.fillna(x.mean()))

    #estimate the  age of Outlet
    df2["Outlet_Establishment_Year"]=2013-df2["Outlet_Establishment_Year"]
    df2 = df2.rename(columns={'Outlet_Establishment_Year': 'Age'})

    #reaname Item_fat_content
    df2["Item_Fat_Content"] = df2["Item_Fat_Content"].replace({"Regular": "reg",
                                                       "LF": "lf",
                                                        "Low Fat":"lf",
                                                        "low fat":"lf"})

    # convert Item_Fat_Content to numeric
    df2["Item_Fat_Content"] = df2["Item_Fat_Content"].replace({"Small":0,"High":1,"Medium":2})
    
    # coverting Item_Type to numeric
    df2["Item_Type"]=df2["Item_Type"].replace(dict(zip(df_grp["Item_Type"],df_grp["index"])))
    df2["Item_Type"]=df2["Item_Type"].astype("int32")
    
    # coverting Outlet_Type to numeric
#     df2["Outlet_Type"]=df2["Outlet_Type"].replace(dict(zip(df_grp_ot["Outlet_Type"],df_grp_ot["index"])))
#     df2["Outlet_Type"]=df2["Outlet_Type"].astype("int32")
    
    df2['Outlet_Size'].fillna(df2['Outlet_Size'].mode()[0], inplace=True)
    #df2["Outlet_Size"] = df2["Outlet_Size"].replace({"Small":0,"High":1,"Medium":2})
    
    df2.drop(["Outlet_Identifier","Item_Identifier"],axis=1,inplace=True)

    # encode categorical features
    df2 = pd.get_dummies(df2,drop_first=True)
    return df2

### Preprocessing

In [16]:
def preprocessing(df2):
    # remove null values in Item_Weight
    df2['Item_Weight']=df2.groupby('Item_Type')['Item_Weight'].apply(lambda x:x.fillna(x.mean()))

    #estimate the  age of Outlet
    df2["Outlet_Establishment_Year"]=2013-df2["Outlet_Establishment_Year"]
    df2 = df2.rename(columns={'Outlet_Establishment_Year': 'Age'})

    #reaname Item_fat_content
    df2["Item_Fat_Content"] = df2["Item_Fat_Content"].replace({"Regular": "reg",
                                                       "LF": "lf",
                                                        "Low Fat":"lf",
                                                        "low fat":"lf"})

    # convert Item_Fat_Content to numeric
    df2["Item_Fat_Content"] = df2["Item_Fat_Content"].replace({"reg":1,"lf":0})

    df2.drop(["Outlet_Identifier","Item_Identifier","Outlet_Size"],axis=1,inplace=True)

    # encode categorical features
    df2 = pd.get_dummies(df2,drop_first=True)
    return df2

In [4]:
df2 = preprocessing2(df)


In [5]:
df2

,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Age,Item_Outlet_Sales,Item_Fat_Content_reg,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.300,0.016047,4,249.8092,14,3735.1380,0,1,0,0,0,1,0,0
1,5.920,0.019278,14,48.2692,4,443.4228,1,1,0,0,1,0,1,0
2,17.500,0.016760,10,141.6180,14,2097.2700,0,1,0,0,0,1,0,0
3,19.200,0.000000,6,182.0950,15,732.3800,1,1,0,0,1,0,0,0
4,8.930,0.000000,9,53.8614,26,994.7052,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0.056783,13,214.5218,26,2778.3834,0,0,0,0,1,1,0,0
8519,8.380,0.046982,0,108.1570,11,549.2850,1,1,0,1,0,1,0,0
8520,10.600,0.035186,8,85.1224,9,1193.1136,0,0,1,1,0,1,0,0
8521,7.210,0.145221,13,103.1332,4,1845.5976,1,1,0,0,1,0,1,0


### Transforming

In [6]:
df2["Item_Outlet_Sales"]=np.sqrt(df2['Item_Outlet_Sales'])


In [7]:
target="Item_Outlet_Sales"
X = df2.drop(target,axis=1) 
y = df2[target]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0)

In [10]:
# scaler = StandardScaler() 
# scaler.fit(X_train) 
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test) 

### Linear Reg

In [9]:
from sklearn.linear_model import LinearRegression 
model = LinearRegression() 
model.fit(X_train,y_train) 

LinearRegression()

### Decision Tree - Grid Search


In [34]:
def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true (y_true) and predicted (y_predict) values based on the metric chosen. """
    
    #score = r2_score(y_true, y_predict)
    score=np.sqrt(mean_squared_error(y_true, y_predict)
    return score

SyntaxError: invalid syntax (Temp/ipykernel_5252/2940909763.py, line 7)

In [35]:
def fit_model(X, y):
    """ Performs grid search over the 'max_depth' parameter for a 
        decision tree regressor trained on the input data [X, y]. """
    
    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 0)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[1,2,3,4,5,6,7,8,9,10]}

    # Transform 'performance_metric' into a scoring function using 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    grid = GridSearchCV(estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    grid = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid.best_estimator_

In [36]:
model=fit_model(X_train, y_train)

### Random Forest


In [50]:
model = RandomForestRegressor(n_estimators= 100, max_depth = 5, max_features='sqrt')
model.fit(X_train,y_train) 

RandomForestRegressor(max_depth=5, max_features='sqrt')

### XG Boost

In [10]:
for lr in [0.03,0.035,0.04,0.045,0.05,0.055,0.06,0.065,0.07,0.08]:
    model = xgb.XGBRegressor(learning_rate = lr, n_estimators=100, verbosity = 0) # initialise the model
    model.fit(X_train,y_train) #train the model
    model.score(X_test, y_test) # scoring the model - r2 squared
    print("Learning rate : ", lr, " Train score : ", model.score(X_train,y_train), " Cross-Val score : ", np.mean(cross_val_score(model, X_train, y_train, cv=10)))

Learning rate :  0.03  Train score :  0.7022623715595051  Cross-Val score :  0.6572291910638557
Learning rate :  0.035  Train score :  0.7172381444231755  Cross-Val score :  0.6660820289421123
Learning rate :  0.04  Train score :  0.7262716903184578  Cross-Val score :  0.6677689468253528
Learning rate :  0.045  Train score :  0.7339087310297499  Cross-Val score :  0.668868635063668
Learning rate :  0.05  Train score :  0.7381437388970675  Cross-Val score :  0.6679142825375599
Learning rate :  0.055  Train score :  0.7448644407269336  Cross-Val score :  0.6665458640306433
Learning rate :  0.06  Train score :  0.7475993454214072  Cross-Val score :  0.6665204236217075
Learning rate :  0.065  Train score :  0.7545014893538523  Cross-Val score :  0.6656721961832262
Learning rate :  0.07  Train score :  0.7574376024055911  Cross-Val score :  0.6647916717691285
Learning rate :  0.08  Train score :  0.767740125136297  Cross-Val score :  0.6618219780696399


In [11]:
model = xgb.XGBRegressor(learning_rate = 0.045, n_estimators=100)
model.fit(X_train,y_train) #train the model
model.score(X_test, y_test) # scoring the model - r2 squared

0.6608307960123188

### XGB -Optuna : Parametric Tuning

In [108]:
import warnings

warnings.filterwarnings("ignore")

def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25,random_state=42)
    param = {
        #'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'random_state': trial.suggest_categorical('random_state', [24, 48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

# df = pd.read_csv('diamonds.csv').drop(['index'],axis=1)
# X = df.drop(['price'],axis=1)
# y = df['price']
#categorical_features_indices = np.where(X.dtypes != np.float)[0]

study = optuna.create_study(sampler=TPESampler(), direction="maximize")
study.optimize(objective, n_trials=5, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    

[I 2022-06-29 16:55:01,526] A new study created in memory with name: no-name-a924e5c6-31f3-49c8-ad96-6f8f8b09d667
[I 2022-06-29 16:55:07,845] Trial 0 finished with value: 10.3886010779522 and parameters: {'lambda': 0.08822071499595764, 'alpha': 4.996067451579491, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.016, 'max_depth': 20, 'random_state': 2020, 'min_child_weight': 134}. Best is trial 0 with value: 10.3886010779522.
[I 2022-06-29 16:55:14,044] Trial 1 finished with value: 10.368473419377933 and parameters: {'lambda': 0.200514915335686, 'alpha': 0.042723613727901955, 'colsample_bytree': 0.4, 'subsample': 0.8, 'learning_rate': 0.009, 'max_depth': 5, 'random_state': 24, 'min_child_weight': 48}. Best is trial 0 with value: 10.3886010779522.
[I 2022-06-29 16:55:27,441] Trial 2 finished with value: 10.318417511905503 and parameters: {'lambda': 3.858564285753639, 'alpha': 3.507601904384952, 'colsample_bytree': 1.0, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 9,

Number of completed trials: 5
Best trial:
	Best Score: 10.40785116138728
	Best Params: 
    lambda: 1.5827814439459829
    alpha: 0.15985779143462803
    colsample_bytree: 0.5
    subsample: 0.5
    learning_rate: 0.016
    max_depth: 13
    random_state: 24
    min_child_weight: 39


In [ ]:
model = xgb.XGBRegressor(**trial.params) # scoring the model - r2 squared

### Prediction

In [10]:
#model.fit(X_train,y_train) 

y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

In [11]:
#sns.displot(y_train_pred)

### Inverse Trans

In [12]:
y_test_pred=(y_test_pred)**2
y_train_pred=(y_train_pred)**2
y_train=(y_train)**2
y_test=(y_test)**2



### Evaluation

In [13]:
#model.score(X_test, y_test) # scoring the model - r2 squared

In [14]:
print('Train R2-SCORE:', metrics.r2_score(y_train,y_train_pred))
print('Test R2-SCORE:', metrics.r2_score(y_test,y_test_pred))


Train R2-SCORE: 0.5854257532536802
Test R2-SCORE: 0.5833260923571624


In [15]:
print('Train RMSE:',np.sqrt(mean_squared_error(y_train,y_train_pred)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test,y_test_pred)))


Train RMSE: 1087.7550591337367
Test RMSE: 1126.4130337287856


### Test Data

In [64]:
df_test =pd.read_csv("test.csv")

In [65]:
testdf2 = preprocessing2(df_test)

In [66]:
testpred = model.predict(testdf2)

In [67]:
dft=pd.DataFrame(df_test[["Item_Identifier","Outlet_Identifier"]])
dft["Item_Outlet_Sales"]=testpred**2


In [68]:
dft

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1606.733421
1,FDW14,OUT017,1403.469881
2,NCN55,OUT010,1174.243497
3,FDQ58,OUT017,2330.709398
4,FDY38,OUT027,5405.048917
...,...,...,...
5676,FDB58,OUT046,2130.860136
5677,FDD47,OUT018,2215.097036
5678,NCO17,OUT045,1744.741396
5679,FDJ26,OUT017,3393.903543


In [69]:
dft.to_csv("out4.csv",index=False)

In [10]:
# Train R2-SCORE: 0.5848267673273038
# Test R2-SCORE: 0.5821400601197964
    
# Train RMSE: 1088.5405816346295
# Test RMSE: 1128.015021298349